In [1]:
import sys
sys.path.append('../')
from src.utils import create_env
from src.network import Net
from src.agent import Agent
from src.trainer import Trainer

In [2]:
import gym
import torch
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
num_envs = 8
n_frames = 5
frame_skip = 5
vec_env, env = create_env(num_envs, n_frames, frame_skip)
writer = SummaryWriter('../logs/trial-1')

env_pool of size 8 and test env initialized.


In [4]:
noisy = True
net = Net(n_frames, noisy)
device = torch.device('cpu')
distribution = 'categorical'
lr = 3e-4
agent = Agent(net, device)
optimizer = torch.optim.Adam(net.parameters(), lr)

network initialized. Noisy: True, number of parameters: 1673340
Net(
  (conv): Sequential(
    (0): Conv2d(5, 16, kernel_size=(8, 8), stride=(4, 4))
    (1): SELU()
    (2): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2))
    (3): SELU()
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): SELU()
  )
  (projector): NoisyLinear(in_features=1344, out_features=512)
  (policy_stream): NoisyLinear(in_features=512, out_features=128)
  (policy_layer): NoisyLinear(in_features=128, out_features=5)
  (value_stream): NoisyLinear(in_features=512, out_features=128)
  (value_layer): NoisyLinear(in_features=128, out_features=1)
)
agent initialized with categorical policy


In [ ]:
num_tests = 1
value_loss = 'huber'
entropy = 0 if noisy else 1e-1
gamma = 0.99
gae_lambda = 0.95
use_gae = True
norm_adv = False
ppo_eps = 0.1
num_ppo_epoch = 5
ppo_batch_size = 20

trainer = Trainer(vec_env, num_envs, env, num_tests, 
                  agent, distribution, device, optimizer, value_loss, 
                  entropy, gamma, gae_lambda, norm_adv, use_gae,
                  ppo_eps, num_ppo_epoch, ppo_batch_size, 
                  writer)

trainer initialized
training parameters:
	 value_loss: huber, gamma: 0.99, entropy_reg: 0
	 use_gae: True, gae_lambda: 0.95, normalize_adv: False
	 ppo_batch_size: 20, ppo_epochs: 5


In [ ]:
warm_up = False
trainer.train(warm_up, 20, 150, 10)

training start
num_epochs: 20, steps_per_epochs: 150, env_steps: 10


epoch_4:  99%|███████████████████████████████▌| 148/150 [12:11<00:09,  4.79s/it]